# 

In [1]:
import networkx as nx

##########################
## cycles
##########################

# cycle basis of the graph
#def cycle_basis(G):
#    return nx.cycle_basis(G, 0)

def cycle_basis(G):
    all_cycles = []
    all_cycles_set = set()
    for node in G.nodes():
        current_cycles = nx.cycle_basis(G, node)
        current_set = [frozenset(c) for c in current_cycles]
        for idx, s in enumerate(current_set):
            if s not in all_cycles_set:
                all_cycles_set.add(s)
                all_cycles.append(current_cycles[idx])
    return all_cycles

# remove cycle if it forms a clique
def reduced_cycle_basis(cycle_basis, max_cliques):
    return [cy for cy in cycle_b for cl in max_cliques if not set(cy).issubset(cl)]

##########################
## cliques
##########################

# cliques with length > 2
def max_cliques(G):
    return sorted([clique for clique in nx.find_cliques(G) if len(clique)>2])

##########################
## "line" path
## similar to bridges
##########################

# return list of node and corresponding neighbors when the number of neighbors is le than n
def _le_n_neigh_nodes(G, n):
    l_nodes = {}
    for node in nx.nodes(G):
        neighbors = list(nx.neighbors(G, node))
        if len(neighbors) <= n:
            l_nodes[node] = neighbors
    return l_nodes

# construct the path where each node as
def _search_line_rec(l_node, path, l_nodes): # faster implementation possible with dp or check presence
    path.append(l_node)
    if l_node in l_nodes:
        for neighbors in l_nodes[l_node]:
            if neighbors not in path:
                _search_line_rec(neighbors, path, l_nodes)
    return path

# return the "line" paths of the graph, n constrain the "path size"
def line_paths(G, n):
    line_paths = []
    l_nodes = _le_n_neigh_nodes(G, n)

    for l_node in list(l_nodes.keys()):
        path = _search_line_rec(l_node, [], l_nodes)
        line_paths.append(path)

    reduced_line_path = list([list(x) for x in set([frozenset(path) for path in line_paths if len(path) > 2])])
    return reduced_line_path


##########################
## Component
##########################

# Generates nodes in each maximal k-edge-connected component in G.
def k_edge_comp(G, k):
    return [comp for comp in sorted(map(sorted, nx.k_edge_components(G, k=2))) if len(comp) > 2]

#A k-component is a maximal subgraph of a graph G that has, at least, node connectivity k: we need to remove at least k nodes
#to break it into more components. k-components have an inherent hierarchical structure because they are nested in terms of connectivity:
#a connected graph can contain several 2-components, each of which can contain one or more 3-components, and so forth.
def k_comp(G):
    return [list(comp[0]) for comp in nx.k_components(G).values()]


##########################
## Star
##########################

def _star_rec(node, visited, layer):
    if layer == 0:
        return
    visited.add(node)
    for neighbors in list(nx.neighbors(G, node)):
        _star_rec(neighbors, visited, layer-1)
    return list(visited)

def star(G, n):
    stars = []
    for node in nx.nodes(G):
        stars.append(_star_rec(node, set(), n+1))
    return stars

In [2]:
import networkx as nx
import random
import math
import os
import json

def supernode_edges(supernode, concepts):
    return [(supernode, n) for n in concepts]

def add_supernode_normal(G, concepts, features):
    for i, concept in enumerate(concepts):
        supernode_name = G.number_of_nodes()
        G.add_node(supernode_name, x=features)
        G.add_edges_from(supernode_edges(supernode_name, concept))

def _select_random_edge_from_cycle(cycle):
    idx = random.randrange(len(cycle))
    if idx == len(cycle)-1:
        return (cycle[idx], cycle[0])
    else:
        return (cycle[idx], cycle[idx+1])

def remove_cycles(G):
    current_cycle_basis = cycle_basis(G)
    while len(current_cycle_basis) != 0:
        cycle = current_cycle_basis[random.randrange(len(current_cycle_basis))]
        (u, v) = _select_random_edge_from_cycle(cycle)
        G.remove_edge(u, v)
        current_cycle_basis = cycle_basis(G)

def add_random_edges(G, n):
    nodes = list(G.nodes())
    nodes_num = len(nodes) - 1
    new_edges = []
    for _ in range(n):
        u = nodes[random.randrange(nodes_num)]
        v = nodes[random.randrange(nodes_num)]
        new_edges.append((u,v))

    G.add_edges_from(new_edges)


# create a dataset to check the loop
def create_dataset_tree_cycle(parent_dir, dataset_name, graph_num, cycle_proportion=0.5, node_num=80, cycle_level=10):
    path = os.path.join(parent_dir, dataset_name)
    os.mkdir(path)
    num_cycle_graph = int(graph_num * cycle_proportion)
    for cycle_graph in range(num_cycle_graph):
        graph_path = os.path.join(path, str(cycle_graph))
        G = nx.random_tree(node_num)
        G.graph['y'] = 1                            # cycle
        add_random_edges(G, cycle_level)
        nx.write_gml(G, graph_path)

    for tree in range(num_cycle_graph, num_cycle_graph + math.ceil(graph_num * (1-cycle_proportion))):
        graph_path = os.path.join(path, str(tree))
        G = nx.random_tree(node_num)
        G.graph['y'] = 0                            # nocycle
        nx.write_gml(G, graph_path)

    info_data = {
            "size": graph_num
            }
    info_path = os.path.join(path, "info.json")
    with open(info_path, 'w') as f:
        json.dump(info_data, f)

In [3]:
import networkx as nx
from torch_geometric.data import Dataset
from torch_geometric.utils import from_networkx
import torch
from typing import List
from distutils.dir_util import copy_tree
import json
import os.path as osp

class Dataset_tree_cycle(Dataset):
    def __init__(self, root, dataset_path, transform=None, pre_transform=None, pre_filter=None, force_reload=False):
        self.dataset_path = dataset_path
        with open(osp.join(dataset_path, "info.json")) as f:
            info_data = json.load(f)
            self.size         = info_data["size"]
            self.cycle_graphs = info_data["cycle_graphs"]
            self.tree_graphs  = info_data["tree_graphs"]
        super().__init__(root, transform, pre_transform, pre_filter, force_reload)

    @property
    def raw_file_names(self) -> List[str]:
        return [str(i) for i in range(self.size)]

    @property
    def processed_file_names(self) -> List[str]:
        return [f'{i}.pt' for i in range(self.size)]

    def download(self) -> None:
       copy_tree(self.dataset_path, self.raw_dir)

    def process(self) -> None:
        for raw_file_name in self.raw_file_names:
            raw_file_path = osp.join(self.raw_dir, raw_file_name)
            G = nx.read_gml(raw_file_path)
            data = from_networkx(G)
            data.y = data.y.unsqueeze(0)

            if self.pre_filter is not None and not self.pre_filter(data):
                continue

            if self.pre_transform is not None:
                data = self.pre_transform(data)

            torch.save(data, osp.join(self.processed_dir, f'{raw_file_name}.pt'))

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'{idx}.pt'))
        return data


In [4]:
dataset = Dataset_tree_cycle("./dataset", "/home/sam/Documents/network/project/dataset/small")

In [5]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Dataset_tree_cycle(1000):
Number of graphs: 1000
Number of features: 1
Number of classes: 2

Data(x=[30, 1], edge_index=[2, 78], y=[1])
Number of nodes: 30
Number of edges: 78
Average node degree: 2.60
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [6]:
import matplotlib.pyplot as plt

def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()

In [7]:
from torch_geometric.utils import to_networkx

#for graph in dataset:
#    G = to_networkx(graph, to_undirected=True)
#    visualize_graph(G, [0 for x in G.nodes()])

In [8]:
import math
from torch_geometric.loader import DataLoader

# mantains the original dataset tree/cycle proportion 
def create_dataloader(dataset, batch_size=10, train_prop=0.7, test_prop=0.2, val_prop=0.1):
    if train_prop + test_prop + val_prop != 1:
        raise Exception("probabilities doesn't sumup to 1")

    cycle_graphs = dataset[dataset.cycle_graphs[0]: dataset.cycle_graphs[1]]
    tree_graphs  = dataset[dataset.tree_graphs[0]: dataset.tree_graphs[1]]

    cycle_graphs = cycle_graphs.shuffle()
    tree_graphs  = tree_graphs.shuffle()

    part1c_end = int(len(cycle_graphs) * train_prop)
    part1t_end = int(len(tree_graphs) * train_prop)

    part2c_end = int(len(cycle_graphs) * (train_prop + test_prop))
    part2t_end = int(len(tree_graphs) * (train_prop + test_prop))

    train_dataset = cycle_graphs[:part1c_end] + tree_graphs[:part1t_end]
    test_dataset  = cycle_graphs[part1c_end:part2c_end] + tree_graphs[part1t_end:part2t_end]
    val_dataset   = cycle_graphs[part2c_end:] + tree_graphs[part2t_end:]

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, val_loader

    

    


In [9]:
train_loader, test_loader, _ = create_dataloader(dataset, train_prop=0.8, test_prop=0.2, val_prop=0.0)

In [10]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print(data.y)
    print()

Step 1:
Number of graphs in the current batch: 10
DataBatch(x=[300, 1], edge_index=[2, 687], y=[10], batch=[300], ptr=[11])
tensor([1, 1, 0, 0, 0, 1, 1, 1, 1, 0])

Step 2:
Number of graphs in the current batch: 10
DataBatch(x=[300, 1], edge_index=[2, 699], y=[10], batch=[300], ptr=[11])
tensor([0, 1, 0, 1, 0, 1, 1, 1, 1, 1])

Step 3:
Number of graphs in the current batch: 10
DataBatch(x=[300, 1], edge_index=[2, 598], y=[10], batch=[300], ptr=[11])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

Step 4:
Number of graphs in the current batch: 10
DataBatch(x=[300, 1], edge_index=[2, 707], y=[10], batch=[300], ptr=[11])
tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 0])

Step 5:
Number of graphs in the current batch: 10
DataBatch(x=[300, 1], edge_index=[2, 731], y=[10], batch=[300], ptr=[11])
tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 0])

Step 6:
Number of graphs in the current batch: 10
DataBatch(x=[300, 1], edge_index=[2, 647], y=[10], batch=[300], ptr=[11])
tensor([0, 1, 0, 0, 0, 1, 1, 0, 0, 1])

Step 7:
Number o

In [14]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=10)
print(model)
print(dataset.num_classes)

GCN(
  (conv1): GCNConv(1, 10)
  (conv2): GCNConv(10, 10)
  (conv3): GCNConv(10, 10)
  (lin): Linear(in_features=10, out_features=2, bias=True)
)
2


In [15]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         print(out.argmax(dim=1))
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 0, 0, 0, 1, 0, 0, 0, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 1, 0, 0,

KeyboardInterrupt: 

In [16]:
from torch_geometric.data import Data, HeteroData
from torch_geometric.transforms import BaseTransform
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.utils import to_networkx
from typing import Union

#@functional_transform('add_supernodes')
class AddSupernodes(BaseTransform):
    def __init__(self, concepts_list) -> None:
        self.concepts_list = concepts_list

    def forward(self, data: Data) -> Data:
        G = to_networkx(data, to_undirected=True, node_attrs=["x"], graph_attrs=["y"])

        # find all the concepts in the graph on the original graph only
        for concept in self.concepts_list:
            concept["concepts_nodes"] = concept["fun"](G, *concept["args"])

        for concept in self.concepts_list:
            add_supernode_normal(G, concept["concepts_nodes"], [1.0])

        data_with_supernodes = from_networkx(G)
        return data_with_supernodes
            
        

In [17]:
concepts_list_ex = [
        {"name": "GCB", "fun": cycle_basis, "args": []}, 
       {"name": "GMC", "fun": max_cliques, "args": []},
       {"name": "GLP2", "fun": line_paths, "args": [2]}
    ]

# ADD supernode

In [18]:
import torch_geometric.transforms as T

In [19]:
dataset.transform = AddSupernodes(concepts_list_ex)

In [20]:
dataset

Dataset_tree_cycle(1000)

In [21]:
dataset[0]

Data(x=[80, 1], edge_index=[2, 814], y=[1])

In [22]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=10)
print(model)

GCN(
  (conv1): GCNConv(1, 10)
  (conv2): GCNConv(10, 10)
  (conv3): GCNConv(10, 10)
  (lin): Linear(in_features=10, out_features=2, bias=True)
)


In [ ]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 002, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 003, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 004, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 005, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 006, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 007, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 008, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 009, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 010, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 011, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 012, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 013, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 014, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 015, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 016, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 017, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 018, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 019, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 020, Train Acc: 0.5000, Test Acc: 0.5000
Epoch: 021, Train Acc: 0.5000, Test Acc:

In [ ]:
print(len(train_loader.dataset))
print(len(test_loader.dataset))